## Imports

In [1]:
import pandas as pd
import folium
import os

## Data Read-in

In [11]:
df = pd.read_csv('Eviction Notices 2023.xlsx - Sheet1.csv')

In [12]:
df['']

,Eviction ID,Address,City,State,Eviction Notice Source Zipcode,File Date,Non Payment,Breach,Nuisance,Illegal Use,...,DELETE - Police Districts,DELETE - Supervisor Districts,DELETE - Fire Prevention Districts,DELETE - Zip Codes,"CBD, BID and GBD Boundaries as of 2017",Central Market/Tenderloin Boundary,"Areas of Vulnerability, 2016",Central Market/Tenderloin Boundary Polygon - Updated,Fix It Zones as of 2018-02-07,Neighborhoods
0,M230001,1000 Block Of Vicente Street,San Francisco,CA,94116,1/3/2023,True,False,False,False,...,8.0,3.0,1.0,29491.0,NaN,NaN,1.0,NaN,NaN,40.0
1,M230002,1000 Block Of Vicente Street,San Francisco,CA,94116,1/3/2023,True,False,False,False,...,8.0,3.0,1.0,29491.0,NaN,NaN,1.0,NaN,NaN,40.0
2,M230003,2900 Block Of California Street,San Francisco,CA,94115,1/4/2023,False,False,True,False,...,6.0,1.0,13.0,29490.0,NaN,NaN,1.0,NaN,NaN,103.0
3,M230004,500 Block Of Octavia Street,San Francisco,CA,94102,1/4/2023,False,False,True,False,...,9.0,11.0,7.0,28852.0,NaN,NaN,1.0,NaN,NaN,22.0
4,M230005,0 Block Of Dolores Street,San Francisco,CA,94103,1/5/2023,False,False,True,False,...,4.0,5.0,8.0,28862.0,9.0,NaN,1.0,NaN,NaN,37.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469,M230796,1900 Block Of 47th Avenue,San Francisco,CA,94116,6/16/2023,False,False,False,False,...,8.0,3.0,1.0,29491.0,NaN,NaN,1.0,NaN,NaN,39.0
470,M230851,3200 Block Of 25th Street,San Francisco,CA,94110,6/30/2023,True,False,False,False,...,4.0,7.0,2.0,28859.0,NaN,NaN,2.0,NaN,NaN,53.0
471,M230852,800 Block Of Ofarrell Street,San Francisco,CA,94109,7/1/2023,False,False,True,False,...,9.0,9.0,7.0,28858.0,NaN,1.0,2.0,1.0,NaN,20.0
472,M230853,600 Block Of Capp Street,San Francisco,CA,94110,7/1/2023,False,True,False,False,...,4.0,7.0,2.0,28859.0,NaN,NaN,2.0,NaN,NaN,53.0


## Make Map

In [8]:
df['File Date'] = pd.to_datetime(df['File Date'])

In [9]:
df = df[df['File Date'] >= '2023-01-01']

In [10]:
df['Eviction ID'].value_counts()

Eviction ID
M230084    5
M230018    5
M230059    5
M230083    5
M230002    5
          ..
M230728    1
M230749    1
M230759    1
M230720    1
M230694    1
Name: count, Length: 474, dtype: int64

In [5]:
df['Location'].isna().value_counts()

Location
False    1380
True       27
Name: count, dtype: int64

In [6]:
df = df.dropna(subset='Location')

In [7]:
def parse_location(location_str):
    # Remove the parentheses and split the string into latitude and longitude
    lat, lon = map(float, location_str[1:-1].split(','))
    return pd.Series([lat, lon], index=['latitude', 'longitude'])

# Apply the function to the 'location' column and join the result with the original DataFrame
coordinates = df['Location'].apply(parse_location)
df = df.join(coordinates)

In [9]:
pd.set_option('display.max_columns',None)

In [16]:
df['True_Count'] = df[['Non Payment', 'Breach',
       'Nuisance', 'Illegal Use', 'Failure to Sign Renewal', 'Access Denial',
       'Unapproved Subtenant', 'Owner Move In', 'Demolition',
       'Capital Improvement', 'Substantial Rehab', 'Ellis Act WithDrawal',
       'Condo Conversion', 'Roommate Same Unit', 'Other Cause',
       'Late Payments', 'Lead Remediation', 'Development']].sum(axis=1)

In [19]:
df['Eviction_Reason'] = df[['Non Payment', 'Breach',
       'Nuisance', 'Illegal Use', 'Failure to Sign Renewal', 'Access Denial',
       'Unapproved Subtenant', 'Owner Move In', 'Demolition',
       'Capital Improvement', 'Substantial Rehab', 'Ellis Act WithDrawal',
       'Condo Conversion', 'Roommate Same Unit', 'Other Cause',
       'Late Payments', 'Lead Remediation', 'Development']].idxmax(axis=1)

In [21]:
df['Eviction_Reason'].value_counts()

Eviction_Reason
Nuisance                460
Capital Improvement     350
Non Payment             216
Breach                  163
Owner Move In            95
Ellis Act WithDrawal     37
Roommate Same Unit       19
Other Cause              16
Unapproved Subtenant     15
Late Payments             5
Access Denial             4
Name: count, dtype: int64

In [23]:
df.columns

Index(['Eviction ID', 'Address', 'City', 'State',
       'Eviction Notice Source Zipcode', 'File Date', 'Non Payment', 'Breach',
       'Nuisance', 'Illegal Use', 'Failure to Sign Renewal', 'Access Denial',
       'Unapproved Subtenant', 'Owner Move In', 'Demolition',
       'Capital Improvement', 'Substantial Rehab', 'Ellis Act WithDrawal',
       'Condo Conversion', 'Roommate Same Unit', 'Other Cause',
       'Late Payments', 'Lead Remediation', 'Development',
       'Good Samaritan Ends', 'Constraints Date', 'Supervisor District',
       'Neighborhoods - Analysis Boundaries', 'Location', 'Shape',
       'SF Find Neighborhoods', 'Current Police Districts',
       'Current Supervisor Districts', 'Analysis Neighborhoods',
       'DELETE - Neighborhoods', 'DELETE - Police Districts',
       'DELETE - Supervisor Districts', 'DELETE - Fire Prevention Districts',
       'DELETE - Zip Codes', 'CBD, BID and GBD Boundaries as of 2017',
       'Central Market/Tenderloin Boundary', 'Areas of Vu

In [25]:
def popup_html(row):
    address = row['Address']
    eviction_reason = row['Eviction_Reason']
    latitude = row['latitude']
    longitude = row['longitude']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Address: </strong>{}'''.format(address) + '''<br>
    <strong>Eviction Reason: </strong>{}'''.format(eviction_reason) + '''<br>
    <strong>Latitude: </strong>{}'''.format(latitude) + '''<br>
    <strong>Longitude: </strong>{}'''.format(longitude) + '''<br>
    </html>
    '''
    return html

In [27]:
# Create a map object
m = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=12)
# Add markers to the map
for i, row in df.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']],
                 popup=folium.Popup(popup_html(row), max_width=400)).add_to(m)
    
# Save it to an HTML file
m.save('index.html')
# Display the map
m

In [28]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/SF_Evictions2023
